In [1]:
from jetcam.csi_camera import CSICamera
import torchvision.transforms as transforms
from xy_dataset import XYDataset
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

In [2]:
model.load_state_dict(torch.load('road_following_model.pth'))

<All keys matched successfully>

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

In [4]:
torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

In [5]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

<All keys matched successfully>

In [6]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

In [7]:
camera = CSICamera(width=224, height=224, capture_fps=65)

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 0.2
STEERING_BIAS = 0.2
car.throttle = 0.1

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS